In [1]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler

from fast_hdbscan.numba_kdtree import kdtree_to_numba
from fast_hdbscan.numba_kdtree import parallel_tree_query
from sklearn.neighbors import KDTree

CPU times: user 11 s, sys: 471 ms, total: 11.5 s
Wall time: 9.09 s


In [51]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

In [52]:
graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

In [53]:
graph.cardinalities.describe()

count    304554.000000
mean          6.751085
std           2.060782
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          82.000000
Name: cardinalities, dtype: float64

In [54]:
from core.cluster_validation import print_distance, generate_neigbhourhood_groups, generate_detailed_clusters

In [55]:
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [142]:
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')



X_train = X_train[X_train.index >= 0]



spatial_lag = 1


# lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/context_chars_{region_id}_lag_{spatial_lag}.parquet')

lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{spatial_lag}.parquet')



X_train = X_train.join(lag[[c for c in lag.columns if '_median' in c]], how='inner')

# X_train = X_train.join(lag, how='inner')


In [143]:
# for c in X_train.columns:
#     X_train[c] = X_train[c].clip(*np.percentile(X_train[c], [5, 95]))

In [144]:
to_drop = ['stcSAl',
 'ltkOri',
 'stbOri',
 'stcOri',
 'stbCeA']


all_drop = []
for c in to_drop:
    all_drop += X_train.columns[X_train.columns.str.contains(c)].tolist()


X_train = X_train.drop(all_drop, axis=1)

In [145]:
vals = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

vals = np.nan_to_num(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)


# X_train = X_train.clip(-10, 10)

In [146]:
# t1 = X_train[[c for c in X_train.columns if '_' not in c]]
# t2 = X_train[[c for c in X_train.columns if '_median' in c]]

# X_train = t1.join(t2)
# X_train.shape

In [147]:
stats = X_train.describe()
stats.columns[stats.loc['std'] == 0]

Index([], dtype='object')

In [148]:
X_train = X_train.drop(stats.columns[stats.loc['std'] == 0], axis=1)

In [149]:
X_train.shape

(299064, 116)

In [150]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(X_train.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(X_train)), index=X_train.index)
    .loc[tess_groups.index]
    .values
)

def check_score(data, example_clusters):
    groups = example_clusters[example_clusters.index.isin(data.index)]
    groups_ilocs = (
        pd.Series(np.arange(len(data)), index=data.index).loc[groups.index].values
    )
    return davies_bouldin_score(data.iloc[groups_ilocs], groups.values)

In [151]:
# tessellation.loc[tess_groups.index].explore()

In [152]:
from core.cluster_validation import print_distance
print_distance( pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean(), metric='euclidean')

,commie blocks vn,fancy commie blocks,holyne,housing blocks,housing houses,josefov,karlin IT offices,karlin old,karlin river offices,karlin square,mala strana,malesice,prague castle,row houses1,row houses2,smickov,stare mesto,vinohrady blocks,vinohrady squares,vinohrady villas
commie blocks vn,0.000000,6.312038,8.452875,6.525083,8.097942,11.013584,7.923148,13.006853,13.254182,8.514979,14.975777,9.729286,12.193692,11.263891,12.864405,10.235305,15.376484,6.999093,11.786976,8.394412
fancy commie blocks,6.312038,0.000000,9.687113,5.349956,9.479605,13.367384,7.194058,13.365348,10.901255,10.544501,15.504024,7.906810,11.126878,12.304027,13.612388,11.649365,16.522064,8.135876,12.772575,9.292153
holyne,8.452875,9.687113,0.000000,8.397234,6.153532,11.876925,9.426268,14.123826,14.432397,10.156774,14.924551,9.325407,11.810953,11.893861,12.690173,11.251768,15.642148,6.525503,12.686189,6.807925
housing blocks,6.525083,5.349956,8.397234,0.000000,7.606639,11.179449,6.720314,12.602258,10.323772,9.420406,14.409193,8.051805,9.846907,12.529436,13.832864,10.039067,14.711283,6.320945,11.926464,7.869856
housing houses,8.097942,9.479605,6.153532,7.606639,0.000000,9.773606,7.748692,12.611583,14.790691,7.773297,15.774995,10.129022,12.932813,10.516855,11.629459,8.778328,15.159375,6.265936,9.306927,3.301848
josefov,11.013584,13.367384,11.876925,11.179449,9.773606,0.000000,12.230159,9.727206,17.751820,5.757432,11.959752,14.381249,11.728982,13.816473,15.112716,4.957961,9.036848,10.982091,7.864355,10.056365
karlin IT offices,7.923148,7.194058,9.426268,6.720314,7.748692,12.230159,0.000000,12.219598,11.676852,9.343825,15.943288,6.488129,12.618378,10.934709,12.285805,9.963637,16.064845,6.040597,11.068242,7.330852
karlin old,13.006853,13.365348,14.123826,12.602258,12.611583,9.727206,12.219598,0.000000,18.185306,7.353770,9.904576,13.423637,10.043085,12.084348,13.219061,6.449043,8.404671,12.302466,7.948481,11.991296
karlin river offices,13.254182,10.901255,14.432397,10.323772,14.790691,17.751820,11.676852,18.185306,0.000000,16.241734,18.982746,10.809709,15.502622,17.903302,18.870138,16.411442,20.171025,11.074500,17.885535,14.429047
karlin square,8.514979,10.544501,10.156774,9.420406,7.773297,5.757432,9.343825,7.353770,16.241734,0.000000,12.494995,11.547378,10.855878,10.294212,11.817116,3.117109,10.684918,8.814721,5.179598,7.741619


In [154]:
from scipy.spatial.distance import pdist
for i, g in X_train.loc[tess_groups.index].groupby(tess_groups.values):
    print(i, np.mean(pdist(g)))

commie blocks vn 9.67052036251904
fancy commie blocks 11.267714905000396
holyne 10.99837908438077
housing blocks 12.930648159823054
housing houses 8.366234640633998
josefov 10.457431319526181
karlin IT offices 11.473961775667338
karlin old 12.51486557172
karlin river offices 17.08040664049104
karlin square 12.032366818449676
mala strana 21.08315083163329
malesice 17.282917980557716
prague castle 23.822979976887634
row houses1 9.298362371269603
row houses2 7.406504319281021
smickov 11.682319462129579
stare mesto 16.78672612873577
vinohrady blocks 9.376098999312095
vinohrady squares 7.786363997213499
vinohrady villas 7.909224329395192


In [19]:
# tessellation.loc[tess_groups.index].explore(column=tess_groups.values, categorical=True)

In [155]:
mean_clusters = pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean()

In [156]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['vinohrady squares']).abs().sort_values(ascending=False)

linP4W_median    2.877075
linP4W           2.875372
linP3W_median    2.756253
linP3W           2.658071
ssbSqu_median    1.944645
                   ...   
ldkAre_median    0.004477
ldkAre           0.003503
ltkWNB_median    0.001734
sdcLAL_median    0.000985
sdbCoA_median    0.000005
Length: 116, dtype: float64

In [157]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['stare mesto']).abs().sort_values(ascending=False)

libNCo_median    4.900441
libNCo           4.866751
ssbERI_median    1.881540
ldbPWL_median    1.836020
ldbPWL           1.788455
                   ...   
sdcLAL           0.011426
sdbAre           0.008415
xcnSCl_median    0.006127
midAre_median    0.000425
ssbCor           0.000406
Length: 116, dtype: float64

In [159]:
from core.utils import used_keys
used_keys['ldbPWL']

'perimeter wall length of adjacent buildings'

In [160]:
# training_data = X_train.loc[tess_groups.index]
# tess_groups_ilocs = (
#     pd.Series(np.arange(len(training_data)), index=training_data.index)
#     .loc[tess_groups.index]
#     .values
# )
# training_data.shape

In [161]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import davies_bouldin_score
from core.cluster_validation import get_linkage_matrix

In [162]:
q1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

# clustering_graph = q1.higher_order(k=3, lower_order=True, diagonal=True).subgraph(X_train.index.values)

clustering_graph = q1.copy()

In [163]:
graph_labels = q1.subgraph(X_train.index.values).component_labels
graph_labels.value_counts()

component labels
776    60555
452    25177
754    17243
99     15583
726    13382
       ...  
247        1
727        1
249        1
250        1
161        1
Name: count, Length: 919, dtype: int64

In [164]:
clustering_graph = clustering_graph.subgraph(graph_labels[graph_labels == 776].index.values)

In [165]:
core_ids = clustering_graph.unique_ids
clustering_graph = clustering_graph.transform('B').sparse

In [166]:
training_data = X_train[X_train.index.isin(core_ids)]

In [167]:


# training_data = training_data[[c for c in training_data.columns if '_' not in c]]


In [168]:
# t1 = training_data[[c for c in training_data.columns if '_' not in c]]
# t2 = training_data[[c for c in training_data.columns if '_median' in c]]

# training_data = t1.join(t2)

In [169]:
# training_data = X_train[X_train.index >=0]

In [170]:
training_data.shape

(60555, 116)

In [171]:
# training_data = training_data[[c for c in training_data.columns if '_' not in c]]
# training_data.shape

In [172]:
%%time
clusterer = AgglomerativeClustering(linkage='ward',
                                    connectivity=clustering_graph, 
                                    # connectivity=q1.subgraph(X_train.index.values).transform('B').sparse, 
                                    compute_full_tree=True,
                                    compute_distances=True)
model = clusterer.fit(training_data)

CPU times: user 1.7 s, sys: 4.72 ms, total: 1.71 s
Wall time: 1.71 s


In [173]:
linkage_matrix = get_linkage_matrix(model)

In [174]:
# fix, ax = plt.subplots(figsize=(40,40))
# # Plot the corresponding dendrogram
# _ = dendrogram(linkage_matrix, truncate_mode="level", p=5, ax=ax)

In [175]:
from sklearn.metrics import calinski_harabasz_score

In [176]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(training_data.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(training_data)), index=training_data.index)
    .loc[tess_groups.index]
    .values
)

In [177]:
for t in range(25, 1000, 25):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    print(t, ' - ', 
          adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
          # (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(training_data, r),
          calinski_harabasz_score(training_data, r)
         )

25  -  0.1472295313319262 2.1179732282110204 78.17535705435589
50  -  0.38541992826231347 2.7988562348374737 157.99111909021607
75  -  0.6171686651455724 3.255957866428771 242.57005699685865
100  -  0.6346504455302648 3.669558002268759 320.4786021828579
125  -  0.6383662724732211 3.9308378176721814 416.29497077705395
150  -  0.6474911466682745 3.802193227300548 523.4077370404005
175  -  0.5887499511695788 3.7599170298646922 613.0429865820141
200  -  0.5316740380090332 3.820280967404827 714.840720729423
225  -  0.5321819453049947 3.8769283121107563 861.0368006831081
250  -  0.4072369965283222 3.975627931698163 1034.4984449024132
275  -  0.41356185089034403 4.210974370921486 1092.2313287723093
300  -  0.36761929359088796 4.623626560241497 1188.857598566695
325  -  0.36761929359088796 4.8803277010764825 1266.230637041317
350  -  0.3665737944790801 5.398234045662284 1474.622598555037
375  -  0.3665737944790801 5.398234045662284 1474.622598555037
400  -  0.38438082400456947 5.82404600176235

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
# for t in range(5, 25, 1):
#     r = fcluster(linkage_matrix, t=t, criterion='distance')
#     # r = pd.Series(r, index=X_train.index)
#     # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
#     print(t, ' - ', 
#           adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
#           # (ssplits > 1).sum() / ssplits.shape[0],
#           davies_bouldin_score(training_data, r),
#           calinski_harabasz_score(training_data, r)
#          )

In [178]:
plotting = tessellation.loc[training_data.index].reset_index()

In [179]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.15 s, sys: 84.4 ms, total: 1.24 s
Wall time: 1.24 s


In [180]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [181]:
from core.cluster_validation import get_color

In [188]:
clusters = fcluster(linkage_matrix, t=75, criterion='distance')

In [189]:
np.unique(clusters, return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [193]:
layer.get_fill_color = get_color(clusters)

In [195]:
new_data = training_data.groupby(clusters).mean()

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,...,sicCAR_median,ldkAre_median,ldkPer_median,lskCCo_median,lskERI_median,lskCWA_median,ltkWNB_median,likWBB_median,sdsAre_median,likWCe_median
-1,0.096838,0.098513,0.215519,0.015326,0.023400,0.004171,0.044517,-0.048403,0.084702,0.020278,...,0.500895,0.057310,0.080978,0.036049,0.061599,0.068517,0.139853,0.334059,-0.081577,0.084435
0,0.458200,1.372845,0.963330,-0.859147,0.890798,0.963945,-1.492571,-0.369359,0.858969,1.266388,...,2.920613,0.657159,0.415072,0.439722,-0.211116,0.175115,-0.161909,1.501148,-0.289282,-0.370828
1,0.334013,0.789217,0.019346,0.246400,0.283390,1.238075,-0.334656,0.357465,0.688257,0.779013,...,2.602981,-0.538652,-0.605401,1.126545,0.910417,-0.700755,1.606055,3.202105,-0.377387,0.192539
2,-0.004438,-0.006340,-0.010715,-0.133622,-0.014503,0.166247,0.070152,-0.181106,-0.000882,0.045409,...,-0.215960,1.400066,1.878044,-1.321441,-1.449925,2.153462,-1.057863,-0.624926,0.322741,-0.601432
3,0.299104,0.869298,0.213079,-0.235526,0.614131,1.041896,-0.706145,0.079218,0.670342,0.835891,...,3.142029,-0.570536,-0.637583,0.772139,0.841402,-0.713095,1.517823,3.561842,-0.413607,0.325108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,-0.107728,-0.253179,-0.018678,0.105953,-0.157494,-0.059727,0.137020,0.009229,-0.269130,-0.207546,...,-0.075559,0.035550,0.249060,-1.080409,-0.849981,0.404008,-0.529737,-0.280210,-0.062237,-0.262502
151,0.011003,0.107360,-0.022390,0.147267,0.195161,0.209517,0.096387,0.101711,0.110055,0.085030,...,-0.071249,0.148090,0.378753,-0.557804,-0.874331,0.464930,-0.333293,-0.230499,-0.134959,-0.516122
152,0.089176,0.128837,-0.022390,0.101813,-0.062957,-0.086174,0.234462,-0.060255,0.205427,0.019850,...,-0.364553,0.155758,0.375488,-0.770281,-1.012223,0.518279,-0.621778,-0.409456,-0.081720,-0.687361
153,-0.019033,0.027164,-0.018054,0.165771,-0.012367,-0.085680,0.114501,0.125100,0.066870,-0.077823,...,-0.104094,0.606276,1.039744,-0.969430,-1.324951,1.133336,-0.475826,-0.265802,-0.277609,-0.558570


In [192]:
# try hdbscan extraction
from fast_hdbscan.boruvka import parallel_boruvka
from fast_hdbscan.cluster_trees import (
    cluster_tree_from_condensed_tree,
    condense_tree,
    extract_eom_clusters,
    get_cluster_label_vector,
    mst_to_linkage_tree,
)
from fast_hdbscan.numba_kdtree import kdtree_to_numba
from sklearn.neighbors import KDTree

condensed_tree = condense_tree(linkage_matrix, 
                               min_cluster_size=175)
cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
selected_clusters = extract_eom_clusters(
    condensed_tree, cluster_tree, allow_single_cluster=False
)
clusters = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

In [191]:
for min_cluster_size in range(25, 500, 25):


    condensed_tree = condense_tree(linkage_matrix, 
                                   min_cluster_size=min_cluster_size)
    cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
    selected_clusters = extract_eom_clusters(
        condensed_tree, cluster_tree, allow_single_cluster=False
    )
    r = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

    print(min_cluster_size, ' - ', 
              adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
              # (ssplits > 1).sum() / ssplits.shape[0],
              davies_bouldin_score(training_data, r),
              calinski_harabasz_score(training_data, r)
             )

25  -  0.16221406246642378 2.9611643421404916 65.75478796621087
50  -  0.33212178482406723 3.448581827787381 105.49572657295741
75  -  0.537505207003703 3.716901383749008 133.408126136966
100  -  0.6129251951580539 4.043852459630953 156.98933270142004
125  -  0.6513718867566624 4.2931946038666275 191.31692021293154
150  -  0.6641707556341199 4.489990500111324 209.8195053362268
175  -  0.665844895469902 4.658188307546151 231.35708341252496
200  -  0.6673581391708368 4.868704170807356 252.95506783885202
225  -  0.6669645356867662 4.925663226430396 268.89845716611114
250  -  0.6791296114521439 5.104744400135985 282.5713051147971
275  -  0.6758736345083602 5.228688931801691 293.7485655316692
300  -  0.6758736345083602 5.348633569857419 312.033469390606
325  -  0.6654334991036134 5.419175475104313 345.2538092359429
350  -  0.6654334991036134 5.475385512886398 363.8254754688778
375  -  0.6353833755463317 5.540794935419785 380.8897498235824
400  -  0.5313256810741013 5.614151811457134 422.999

In [ ]:
for t in range(25, 1000, 25):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    

In [157]:
from core.cluster_validation import get_feature_importance
from core.utils import used_keys

In [69]:
clusters_subset = [11597, 11615, 17742]
clusters_subset = np.where(np.isin(clusters, clusters[clusters_subset]))

In [70]:
imps = get_feature_importance(training_data.iloc[clusters_subset], clusters[clusters_subset])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [72]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_48,cluster_66,cluster_69
0,linWID_higher,lcdMes_higher,ssbCCD_lower
1,ldsMSL_lower,linPDE,ltkOri_lower
2,lddNDe,linP3W_higher,ssbSqu_lower
3,ltkOri_lower,mtbAli_median,lcdMes_median
4,ldsMSL,lskCWA_lower,linWID_higher
5,ldsMSL_median,lskERI_median,sicCAR_lower
6,midRea_median,ltkWNB_median,lskCWA_lower
7,ldkAre_higher,linWID_lower,stcOri_lower
8,lskCCo_lower,mtbNDi_higher,ssbSqu_median
9,ssbCCD_lower,linPDE_lower,sdcLAL_median


In [77]:
used_keys['ltkOri']

'orientation of enclosure'

In [112]:
imps = get_feature_importance(training_data, clusters)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 co

In [119]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,ldkPer,lcdMes,libNCo,lskCWA,ldsAre,ltkWNB,likWBB
1,ltkWNB,likWBB,likWBB,likWBB,midAre,ldkPer,lcdMes
2,lskCCo,ltkWNB,sdsSPO,ltkWNB,ltkOri,lskCCo,ltkWNB
3,ltkOri,lskERI,ldbPWL,lskCCo,linP4W,lskCWA,linWID
4,lskCWA,lskCWA,ltcBuA,ldkAre,sdsSPO,ltkOri,ltkOri
5,ldkAre,ldkAre,sicCAR,lskERI,ldsMSL,likWBB,lskCWA
6,likWBB,linP4W,mtbSWR,ldsAre,lddNDe,ldkAre,ldkAre
7,lcdMes,ldkPer,ssbSqu,lddNDe,lcdMes,lskERI,lddNDe
8,lskERI,ltkOri,lskERI,midRea,ldkPer,linP3W,linP3W
9,linP4W,linPDE,sdbPer,linPDE,likWBB,ldsAre,lcnClo


In [122]:
used_keys['lcdMes']

'local meshedness of street network'

In [ ]:
imps[[c for c in imps.columns if '_vals' in c]].cumsum(axis=1)

In [96]:
josefov_joins = []

josefov_joins.append(np.isin(linkage_matrix[:, 0], 
                             tess_groups_ilocs[tess_groups == 'josefov']))
josefov_joins.append(np.isin(linkage_matrix[:, 1], 
                             tess_groups_ilocs[tess_groups == 'josefov']))


In [97]:
indxs = linkage_matrix[josefov_joins[0] | josefov_joins[1]]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs <= X_train.shape[0]]

In [150]:
indxs = linkage_matrix[linkage_matrix[:, 2] <= 2]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs < X_train.shape[0]]
indxs.shape

(64349,)

In [151]:
plotting = tessellation.loc[X_train.iloc[indxs].index]

In [42]:
cluster_means = training_data.groupby(clusters).mean()

In [68]:
c1 = 6
c2 = 10

(cluster_means.loc[c1] - cluster_means.loc[c2]).sort_values(ascending=False)

libNCo           6.284668
libNCo_median    5.846288
libNCo_higher    4.233946
linPDE_higher    3.014914
linPDE           2.122206
                   ...   
lcnClo          -1.414220
linWID_lower    -1.422183
linP3W          -1.489277
lcnClo_lower    -1.702326
linP3W_lower    -2.339139
Length: 248, dtype: float64

In [72]:
from core.utils import used_keys
used_keys['libNCo']

'number of courtyards within adjacent buildings'

In [169]:
bgraph = read_parquet(graph_dir + f"building_graph_{region_id}_knn1.parquet")

In [170]:
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'

buildings = gpd.read_parquet(
        buildings_dir + f"buildings_{region_id}.parquet"

)

In [178]:
buildings

,index,id,geometry
0,0,v0.1-CZE.12.2_1-35164,"POLYGON ((4614847.626 2975218.938, 4614848.235..."
1,1,v0.1-CZE.12.2_1-35123,"POLYGON ((4615276.357 2976034.184, 4615282.866..."
2,2,v0.1-CZE.12.2_1-35159,"POLYGON ((4615315.503 2975986.2, 4615322.056 2..."
3,3,v0.1-CZE.12.2_1-35166,"POLYGON ((4615222.339 2976016.91, 4615224.582 ..."
4,4,v0.1-CZE.12.2_1-35228,"POLYGON ((4615300.348 2975924.258, 4615301.6 2..."
...,...,...,...
299059,299060,v0.1-CZE.13.3_1-13696,"POLYGON ((4618611.169 3033535.197, 4618623.01 ..."
299060,299061,v0.1-CZE.13.3_1-13674,"POLYGON ((4618611.989 3033568.153, 4618617.119..."
299061,299062,v0.1-CZE.13.3_1-13591,"POLYGON ((4618614.831 3033550.704, 4618628.289..."
299062,299063,v0.1-CZE.13.3_1-13328,"POLYGON ((4618625.628 3033512.926, 4618625.634..."


In [181]:
buildings = buildings.join(X_train, how='inner').drop(['index', 'id'], axis=1)

In [185]:
r = buildings.dissolve(bgraph.component_labels, aggfunc='mean')

In [189]:
plotting = r

In [190]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.65 s, sys: 178 ms, total: 1.83 s
Wall time: 1.82 s


In [191]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [156]:
enclosures = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/enclosures/enclosure_{region_id}.parquet")
encl_counts = tessellation.groupby('enclosure_index').count()
encl_counts.columns = ['tessellation']
enclosures['lieWCe'] = encl_counts['tessellation'] / enclosures.geometry.area

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Length: 15958, dtype: float64

In [251]:
enclosures['lieWCe'] 

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Name: lieWCe, Length: 15958, dtype: float64

In [164]:
encl_counts['tessellation']

enclosure_index
7        199
8         52
11         1
12        25
14       962
        ... 
25157      9
25158      1
25159     13
25160     15
25161      3
Name: tessellation, Length: 15935, dtype: int64

In [ ]:
# Measure weighted cells within enclosure
merged = enclosures[['eID', 'ldeAre']].merge(encl_counts[['tessellation']], how='left', on='eID')
enclosures['lieWCe'] = merged['tessellation'] / merged['ldeAre']